In [12]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


### Prepare Data

In [13]:
import pandas as pd
import numpy as np

In [14]:
numeric_cols = [
    "price",
    "usable_areas",
    "total_areas",
    "parking_spaces",
    "suites",
    "bathrooms",
    "bedrooms",
]

In [31]:
data_size = 9100
data = pd.read_csv(f"../data/raw_data/raw_data_{data_size}.csv")

In [32]:
data.rename(
    columns={
        "listing.pricingInfo.rawPrice": "price",
        "link.data.city": "city",
        "link.data.state": "state",
        "link.data.neighborhood": "neighborhood",
        "listing.usableAreas": "usable_areas",
        "listing.totalAreas": "total_areas",
        "listing.parkingSpaces": "parking_spaces",
        "listing.suites": "suites",
        "listing.bathrooms": "bathrooms",
        "listing.bedrooms": "bedrooms",
    },
    inplace=True,
)
cols_to_select = [
    "price",
    "city",
    "state",
    "neighborhood",
    "usable_areas",
    "parking_spaces",
    "suites",
    "bathrooms",
    "bedrooms",
]

numeric_cols = [
    "price",
    "usable_areas",
    "parking_spaces",
    "suites",
    "bathrooms",
    "bedrooms",
]

# Specify the columns to remove outliers from
cols_to_clean = [
    "price",
    "usable_areas",
]

data = data[cols_to_select]
data = data.dropna()
for col in numeric_cols:
    data[col] = data[col].astype(int)
# Define the lower and upper bounds for filtering outliers
quantiles = data[cols_to_clean].quantile([0.05, 0.95])
lower_bound = quantiles.loc[0.05]
upper_bound = quantiles.loc[0.95]

# Filter out the outliers from each specified column
for col in cols_to_clean:
    data = data[(data[col] > lower_bound[col]) & (data[col] < upper_bound[col])]

In [33]:
# Create a new column with  prices to categorical

data["price_cat"] = pd.cut(
    data["price"], bins=[0, 500000, 1000000, np.inf], labels=[1, 2, 3]
)

In [ ]:
data["neighborhood"] = data["city"] + ", " + data["neighborhood"]

In [34]:
data["neighborhood_mean_price_per_m2"] = data["price"] / data["usable_areas"]

neighborhood_median_price_per_m2 = data.groupby("neighborhood")[
    "neighborhood_mean_price_per_m2"
].median()

data["neighborhood_mean_price_per_m2"] = data["neighborhood"].map(
    neighborhood_median_price_per_m2
)

In [35]:
data["price_per_m2"] = data["price"] / data["usable_areas"]

In [39]:
neighborhood_counts = data["formatted_neighborhood"].value_counts()

print(neighborhood_counts)

formatted_neighborhood
Fortaleza, Meireles                         478
Fortaleza, Aldeota                          438
Fortaleza, Cocó                             345
Fortaleza, Engenheiro Luciano Cavalcante    225
Aquiraz, Porto das Dunas                    221
                                           ... 
Caucaia, Tabuba                               1
Caucaia, Planalto Caucaia                     1
Eusébio, Mosquito                             1
Pindoretama, Centro                           1
Juazeiro do Norte, Frei Damião                1
Name: count, Length: 285, dtype: int64


In [19]:
from sklearn.preprocessing import StandardScaler

# Selecting the columns to standardize
columns_to_standardize = ["usable_areas"]

# Creating a StandardScaler object
sc = StandardScaler()

# Fitting the StandardScaler object to the selected columns
data[columns_to_standardize] = sc.fit_transform(data[columns_to_standardize])

In [21]:
data.to_csv(f"../data/processed_data/clean_data_{len(data)}.csv", index=False)

7292

### Load ZAP dataset from Kaagle and do a inflation adjustment

In [23]:
# dados do dataZAP filtrado deixando apenas as casas que são de vendas e retirando as que são para alugel
data_zap = pd.read_csv(
    "../data/raw_data/dataZAP.csv",
    encoding="utf-8",
    sep=";",
    thousands=".",
    decimal=",",
)

/tmp/ipykernel_72766/2546126000.py:2: DtypeWarning: Columns (22,75) have mixed types. Specify dtype option on import or set low_memory=False.
  data_zap = pd.read_csv(
